In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as spl
from scipy.spatial.transform import Rotation
%matplotlib inline

In [2]:
def checkTypes(phaseVec):
    if not isinstance(phaseVec, np.ndarray):
        print("The phase vector must be a numpy array.")
        return False
        return False
    if not phaseVec.shape[0] == 6:
        print("The phase vector must be a 6xN matrix, where N is the number of particles")
        return False
    return True

def newtonsForceLaw(phaseVec, GM=1): # takes in a phaseVec 6xN and outputs a force 3xN, m=1 always
    norms = np.linalg.norm(phaseVec[:3,], axis = 0)
    return -phaseVec[:3,]/np.power(norms, 3)

def turn(phaseVec, n=1, timestep=0.001):
    if not checkTypes(phaseVec):
        return
    newPhaseVec =  np.empty((6, phaseVec.shape[1]))
    force = newtonsForceLaw(phaseVec)
    
    newPhaseVec[:3,] = phaseVec[:3,] + timestep*phaseVec[3:,]
    newPhaseVec[3:,] = phaseVec[3:,] + timestep*force
    return newPhaseVec

In [3]:
class OrbitType:
    ELLIPS = 0
    HYPER = 1

def energyCart(phaseVec, GM=1):
    norms = np.linalg.norm(phaseVec[:3,], axis = 0)
    return np.square(np.linalg.norm(phaseVec[3:,], axis=0))/2 - GM/norms

def angular_momentum_norm_spherical(phaseVec):
    return np.sqrt(np.square(phaseVec[4,]/phaseVec[0,]) + np.square(phaseVec[5,]/(phaseVec[0,]*np.sin(phaseVec[1,]))))
    
def energySph(phaseVec, GM=1):
    return np.square(phaseVec[3,])/2 + np.square(angular_momentum_norm_spherical(phaseVec))/2 - GM/phaseVec[0,]

def angular_momentum(phaseVec):
    return np.cross(phaseVec[:3,], phaseVec[3:,], axisc=0)

def radius(phaseVec):
    return np.linalg.norm(phaseVec[:3,], axis=0)

def spherical_angles(phaseVec):
    return np.arctan(phaseVec[1,]/phaseVec[0,]), np.arctan(phaseVec[2,]/np.sqrt(np.square(phaseVec[1,]) + np.square(phaseVec[0,])))
    
def findOribtalPerametersCartisian(phaseVec, GM = 1, time = 0):
    L = angular_momentum(phaseVec)
    L_2 = np.square(np.linalg.norm(L, axis=0))
    L_z = L[2,]
    E = energyCart(phaseVec)
    
    ORBIT_TYPE = E >= 1
    
    # Action Perameters
    
    a = -1/(2*E)
    
    e = np.sqrt(1 - L_2/a)
    
    theta_0 = np.arcsin(L_z/np.sqrt(L_2))
    
    # Initial Action Perameters
    
    Ellistic_anomily = np.arccos((1 - radius(phaseVec)/a)/e)
    
    M_0 = Ellistic_anomily - e*np.sin(Ellistic_anomily)
    
    f = 2*np.arctan(np.sqrt((1 + e)/(1 - e)) * np.tan(Ellistic_anomily/2))
    
    theta, phi = spherical_angles(phaseVec)
    
    unit_phi = np.array([np.cos(phi)*np.cos(theta), np.cos(phi)*np.sin(theta), np.sin(phi)])
    
    theta_sign = np.sign(np.einsum('ij,ij->j', phaseVec[3:,], unit_phi))
    
    Arg_Peri = -f + theta_sign*np.arcsin(np.cos(theta)/np.cos(theta_0))
    
    Long_of_Ascend = np.sign(L_z)*phi - theta_sign*np.arcsin(np.tan(theta_0)/np.cos(theta))
    
    T_0 = time
    
    return ORBIT_TYPE, a, e, theta_0, M_0, Arg_Peri, Long_of_Ascend, theta_sign, T_0
    
def findOribtalPerametersSpherical(phaseVec, GM = 1, time = 0):
    # vector order [r, theta, phi, r_prime, theta_prime, phi_prime]
    L_2 = np.square(angular_momentum_norm_spherical(phaseVec))
    L_z = phaseVec[4,]
    E = energy(phaseVec)
    
    ORBIT_TYPE = E >= 1
    
    # Action Perameters
    
    a = -1/(2*E)
    
    e = np.sqrt(1 - L_2/a)
    
    theta_0 = np.arcsin(L_z/np.sqrt(L_2))
    
    # Initial Action Perameters
    
    Ellistic_anomily = np.arccos((1 - phaseVec[0,]/a)/e)
    
    M_0 = Ellistic_anomily - e*np.sin(Ellistic_anomily)
    
    f = 2*np.arctan(np.sqrt((1 + e)/(1 - e)) * np.tan(Ellistic_anomily/2))
    
    theta_sign = np.sign(phaseVec[4,])
    
    Arg_Peri = -f + theta_sign*np.arcsin(np.cos(phaseVec[1,])/np.cos(theta_0))
    
    Long_of_Ascend = np.sign(L_z)*phaseVec[2,] - theta_sign*np.arcsin(np.tan(theta_0)/np.cos(phaseVec[1,]))
    
    T_0 = time
    
    return ORBIT_TYPE, a, e, theta_0, M_0, Arg_Peri, Long_of_Ascend, theta_sign, T_0
    
def findPhaseVec(time, ORBIT_TYPE, a, e, theta_0, M_0, Arg_Peri, Long_of_Ascend, theta_sign, T_0):
    
    orbit_period = np.power(a, 3/2)
    

In [5]:
phaseVec = [[1.0],[0],[0],[0],[.01],[0]]
print(findOribtalPerametersCartisian(phaseVec))

TypeError: list indices must be integers or slices, not tuple